In [1]:
import os

In [2]:
%pwd

'/home/tousside/Documents/recrutement/cowrywise-customer-plan-abandonment/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tousside/Documents/recrutement/cowrywise-customer-plan-abandonment'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/koomi/cowrywise-customer-plan-abandonment.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="koomi"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b918e1b0fdebad41476188587e7a2996300c6ee2"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: str
    target_column: str
    mlflow_uri: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories,save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/koomi/cowrywise-customer-plan-abandonment.mlflow",
           
        )

        return model_evaluation_config

In [15]:
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score
)

import os
import pandas as pd
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from urllib.parse import urlparse

In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self, model,  X_text, y_test):
        preds = model.predict(X_text)
        predict_probs = model.predict_proba(X_text)[:, 1]
        accuracy = accuracy_score(y_test, preds)
        roc_auc = roc_auc_score(y_test, predict_probs)
        
        return accuracy, roc_auc
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]
        categorical_cols = ["gender_id", "type"]
        
        X_test[categorical_cols] = X_test[categorical_cols].astype(str)
        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():


            (accuracy, roc_auc) = self.eval_metrics(model,  X_test, y_test)
            
            # Saving metrics as local
            scores = {"accuracy": accuracy, "roc_auc": roc_auc}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("roc_auc", roc_auc)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBoostModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    

In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-29 16:31:13,140: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-29 16:31:13,153: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-29 16:31:13,165: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-29 16:31:13,167: INFO: common: created directory at: artifacts]
[2025-05-29 16:31:13,170: INFO: common: created directory at: artifacts/model_evaluation]


[2025-05-29 16:31:14,171: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/05/29 16:31:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGBoostModel'.
2025/05/29 16:31:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostModel, version 1
Created version '1' of model 'XGBoostModel'.


🏃 View run incongruous-fish-189 at: https://dagshub.com/koomi/cowrywise-customer-plan-abandonment.mlflow/#/experiments/0/runs/3cae80a3970e454e84128fc55bc59f25
🧪 View experiment at: https://dagshub.com/koomi/cowrywise-customer-plan-abandonment.mlflow/#/experiments/0
